#### I need to provide embedding ids for each userid in the train set and have that same embedding id for that same user in the test set dataframe.  Since their are ~931,000 users in the final month of the train set and ~929,000 in the train set, lets first make sure that all of the test users are in the train set so that I could generate the embedding ids in the train set and those to the test dataframe with a simple join.

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = train.loc[train['fecha_dato']=='2016-05-28']

In [4]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
test.ncodpers.isin(train.ncodpers).sum()

929615

In [6]:
test.ncodpers.nunique()

929615

#### The two numbers above are the same so I can proceed with the join approach


In [7]:
test['user_emb_id'] = test['ncodpers'].astype('category').cat.codes

In [8]:
joindf = test[['ncodpers','user_emb_id']]

In [9]:
train = train.merge(joindf,how='right',on='ncodpers')

In [10]:
test.user_emb_id.max()

929614

In [11]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [12]:
labels = train[product_cols]

In [13]:
labels['user_emb_id'] = train.user_emb_id

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
labels = labels.set_index("user_emb_id")

In [15]:
stacked_labels = labels.stack()

In [16]:
filtered_labels = stacked_labels.reset_index()

In [17]:
filtered_labels.columns = ["user_emb_id", "product", "interaction"]

In [18]:
filtered_labels['product'] = filtered_labels['product'].astype('category').cat.codes

In [19]:
filtered_labels.head()

,user_emb_id,product,interaction
0,310071,0,0.0
1,310071,1,0.0
2,310071,2,0.0
3,310071,3,0.0
4,310071,4,0.0


In [20]:
interactions = filtered_labels.loc[filtered_labels['interaction']==1]

In [21]:
interactions.head()

,user_emb_id,product,interaction
26,310140,2,1.0
50,310144,2,1.0
76,310142,4,1.0
84,310142,12,1.0
87,310142,15,1.0


In [22]:
print(test.user_emb_id.nunique())
print(interactions.user_emb_id.nunique())

929615
696467


In [23]:
user_ids = interactions['user_emb_id'].values
items_ids = interactions['product'].values

In [24]:
test_users = interactions['user_emb_id'].drop_duplicates().values

In [25]:
len(test_users)

696467

In [34]:
test_users.max()

929614

#### So of the ~929k  users I need to generate predictions for almost 700k have latent feedback data in the last month of the training data.  Spotlight doesn't work for users without any latent or explicit feeback data.  For the users which matrix factorization method do not apply I wil recommend them the 7 most common products.

In [26]:
from spotlight.interactions import Interactions

In [27]:
interactionmatrix = Interactions(user_ids,items_ids)

In [28]:
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [29]:
train, test = random_train_test_split(interactionmatrix)

In [30]:
model = ImplicitFactorizationModel(n_iter=3,loss='bpr',use_cuda=True)
model.fit(train,verbose=True)

Epoch 0: loss 0.1612430347116731
Epoch 1: loss 0.11481227971630703
Epoch 2: loss 0.09631639530486842


In [31]:
test_users = joindf['user_emb_id'].values

In [33]:
test_users.max()

929614

In [32]:
pred = model.predict(test_users)

ValueError: Maximum user id greater than number of users in model.